In [ ]:
import numpy as np
import gzip
import os
import tensorflow as tf
import re

# specify the path to the zipped file.
os.chdir('/mnt/c/Users/amsj1/OneDrive - University of Tennessee/2nd_year/BZAN554_deep_learning/bzan554-group-assignment1')

In [ ]:
# function creation cell

def parse(path):
    """
    Function to read in the VERY LARGE dataset and yield it as a generator for
    memory efficiency. Takes one argument which is the path to the file being
    read in. This path is set abose using 'os.chdir'.
    """
    with gzip.open(path, 'rb') as g:
        for l in g:
            yield eval(l)


def get_data():
    counter = 0
    X = []
    Y = []
    for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'):
        counter += 1
        X.append(d['title'])
        Y.append(d['category'])
        if counter == 10:
            break

    return X, Y

# get X and Y data (only 100k records)
data_100k = get_data()

In [ ]:
len(data_100k[0])

In [ ]:
#tokenize/split X (titles)
X = []
for titles in data_100k[0]:
    # split the titles into words and lowercase
    words = titles.lower().split()
    for word in words:
        # extract out only words with minimal to no symbols
        stripped = re.sub('[0-9,.,#,\-,&,;,\',(,),/,@,+,!]', '', word)
        X.append(stripped)

X = np.array(X)

# flatten, & uniquefy Y (categories)
Y = np.array(data_100k[1])
flat_Y = [item for subcat in Y for item in subcat]
unique_Y = np.unique(np.array(flat_Y))

# make lookup table with index value for each category
indices = np.array(range(len(unique_Y)), dtype = np.int64)
lookuptable = np.column_stack([unique_Y,indices])

# apply lookup table to data
main_result = []
for i in range(len(Y)):
    res = []
    for ii in range(len(Y[i])):
        res.append(int(lookuptable[lookuptable[:,0]==Y[i][ii],1][0]))
    main_result.append(res)

# create dummy encoded data
y_final = np.array([list(np.zeros(len(unique_Y))) for i in range(len(Y))])
for i in range(len(Y)):
    for ii in range(len(main_result[i])):
        y_final[i,main_result[i][ii]] = 1

y_final

In [ ]:
# set up model architecture
inputs = tf.keras.layers.Input(shape = (1,))
hidden1 = tf.keras.layers.Dense(
    units = 2,
    activation = 'sigmoid',
    name = 'hidden1'
)(inputs)
hidden2 = tf.keras.layers.Dense(
    units = 2,
    activation = 'sigmoid',
    name = 'hidden2'
)(hidden1)
output = tf.keras.layers.Dense(
    units = len(data_100k[1]),
    activation = 'softmax',
    name = 'output'
)(hidden2)

# create & compile the model
model = tf.keras.Model(inputs = inputs, outputs = output)
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.001)
)

In [ ]:
for i in range(1000): # number of epochs
    for j in yield_columns(): # number of instances
        model.fit(
            x=j[0],
            y=j[1],
            epochs=1,
            batch_size=1
        )